In [1]:
#integrate all properly
setwd("/staging/leuven/stg_00041/Bradley/")


In [2]:
library(umap)
library(ggplot2)
library(Seurat)
library(reshape2)

Attaching SeuratObject



In [8]:
#rivron
rivronLoaded <- read.table("/staging/leuven/stg_00041/Bradley/rivronData/rivronBlastoids.hg38.counts.tsv", h=T, sep="\t")
rivronCellMeta <- read.table("/staging/leuven/stg_00041/Bradley/rivronData/rivron.metadata.tsv", h=T, sep="\t")

rivronCellMeta$type <- NA
for(i in 1:nrow(rivronCellMeta)){
    rivronCellMeta$type[i]<- strsplit(as.character(rivronCellMeta$title[i]), split="-")[[1]][1]
}

In [13]:
#petropolous
petropolous <- read.table("/staging/leuven/stg_00041/Published_Embryo_Data/Petropoulos_preImplantation//Mapped_hg38/CountMatrix//ESC_raw_counts_Pet.csv", h=T, sep=",")

#1st column is genes
rownames(petropolous) <- petropolous$X
petropolous <- petropolous[,-1]



In [14]:
#cell meta from colnames

petropolousCellMeta <- read.table("/staging/leuven/stg_00041/Published_Embryo_Data/Petropoulos_preImplantation/Mapped_hg38/Annotation/annotation_Pet.csv", h=T, sep=",")
petropolousCellMeta <- merge(data.frame(colnames(petropolous)), petropolousCellMeta, by.x=1, by.y=8)


petropolousCellMeta <- merge(petropolousCellMeta, zhouPetReannotate, by.x=1 , by.y=1 , all.x=T)


In [23]:
#tyser
#https://doi.org/10.1101/2020.07.21.213512

tyser <- read.table("/staging//leuven/stg_00041/Published_Embryo_Data/Tyser_Gastrulation/Tyser_counts_Gael.tsv", h=T)
tyserCellAnnotation <- read.table("/staging//leuven/stg_00041/Published_Embryo_Data/Tyser_Gastrulation/Tyser_annotation_Gael.tsv", h=T, sep="\t")

rownames(tyser)<- tyser$rownames.dat.

tyser <- t(tyser[,-1])

In [24]:
#Pasque data
#from Xuan Ai, scRNA-seq of the naive, primed, TSC and EXM

pasque <- read.table("/lustre1/project/stg_00041/XuanAi/2020_2021/Naive_primed_TSCs/Naive_Primed_TSCs_countMtx_Ai_mouse_20210624.csv", h=T, sep=",")



In [25]:
rownames(pasque) <- pasque[,1]
pasque <- pasque[,-1]

In [42]:
#making a combined meta-file
combinedMeta <- data.frame(study=c(
                                  rep.int("rivron", times=ncol(rivronLoaded)),
                                  rep.int("petropolous", times=ncol(petropolous)),
                                  rep.int("tyser", times=ncol(tyser)),
                                  rep.int("pasque", times=ncol(pasque))),#for making a publication figure, probably want to use first author for all
                          
                           GEO=c(rep.int("GSE177616", times=ncol(rivronLoaded)),
                                rep.int(NA, times=ncol(petropolous)),
                                rep.int(NA, times=ncol(tyser)), 
                                rep.int(NA, times=ncol(pasque))),
                           
                           
                          sampleID=c(rivronCellMeta$type,
                                    petropolousCellMeta$Embryo_ID, 
                                    tyserCellAnnotation$origin,
                                    rep.int("pasque", times=ncol(pasque))),
                           
                          time=c(rivronCellMeta$characteristics..treatment,
                                petropolousCellMeta$Stage,
                                 rep.int("D16-19", times=ncol(tyser)),
                                 rep.int(NA, times=ncol(pasque))
                                ),
                           cell=c(colnames(rivronLoaded),
                                 colnames(petropolous),
                                 tyserCellAnnotation$rownames.dat.,
                                 colnames(pasque)),
                           cellLineageOriginal=c(#the embryonic datasets have nice cell lineage predictions, blastoids dont appear to
                           
                               rivronCellMeta$type, #I think this is the best I got? it isnt really cell lineage...
                               petropolousCellMeta$finalCluster, tyserCellAnnotation$cluster,
                               rep.int(NA, times=ncol(pasque)))
                        
                          )



In [43]:
combinedMeta$cellLineage <- combinedMeta$cellLineageOriginal

combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("Undefined", "unknown"))]<- "NA"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("Trophectoderm", "TE", "Early Trophectoderm", "early_TE", "late_TE", "medium_TE", "ysTE"))]<- "TE"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("EPI", "EPI_PE", "Epiblast", "PSA-EPI"))]<- "EPI"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("ICM", "Inner Cell Mass", "Inner Cell Mass-Trophectoderm Transition"))]<- "ICM"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("blastoid_24h", "blastoid_60h_PDGFRa_pl",
                                                            "blastoid_60h_TROP2_min",
                                                            "blastoid_60h_TROP2_pl",
                                                            "blastoid_96h_DN",
                                                            "blastoid_96h_PDGFRa_pl",
                                                            "blastoid_96h_TROP2_pl",
                                                            "XMU_MP_1",
                                                            "SC144"))]<- "blastoid"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("naiveWIBR3", "naive_H9"))]<- "naive"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("primed_H9"))]<- "primed"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("Undefined", "unknown", "Unknown","Transitioning", "MIX", "NA", NA))]<- "unknown"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("okae_bts5", "okae_bts5_3D"))]<- "TS"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("Hypoblast"))]<- "hypo"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("naive.postBlastoid",
                                                            "naive.postBlastoid.extraEmbryonicEndoderm",
                                                            "postBlastoid.TSC"))]<- "postBlastoid"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("Advanced Mesoderm", "Axial Mesoderm", "Emergent Mesoderm",
                                                            "Nascent Mesoderm", "YS Mesoderm"))] <- "mesoderm"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("TB.apoptosis", "TB.early", "TB.late", "TB.medium1", "TB.medium2", "TB.medium3" , "TS"))]<- "TB"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("EVT", "EVTs", "Pre.EVT"))]<- "EVT"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("Erythrocytes", "Hemogenic Endothelial Progenitors"))]<- "erythrocytes"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("ST", "STBs", "Pre.ST"))]<- "STB"
combinedMeta$cellLineage[which(combinedMeta$cellLineage%in%c("Endoderm"))]<- "PrE"


options(width=120)

table(combinedMeta$cellLineage, combinedMeta$study, useNA="always")

                  
                     guo iBlastoid junWu pasque petropolous rivron sozen tyser xiang  zhou  <NA>
  B1_B2                0         0     0      0         174      0     0     0     0     1     0
  CTBs                 0         0     0      0           0      0     0     0   149     0     0
  EPI                104         0     0      0         163      0     0   133   165   283     0
  EVT                  0         0     0      0           0      0     0     0    29   464     0
  Ectoderm             0         0     0      0           0      0     0    29     0     0     0
  EightCells           0         0     0      0          83      0     0     0     0     0     0
  ICM                 45         0     0      0           0      0     0     0    52     0     0
  Morula               0         0     0      0         149      0     0     0     0     0     0
  PrE                  0         0     0      0          26      0     0   135     0   164     0
  Primitive

In [13]:
save(combinedMeta, file="combinedMeta.Robject")

In [ ]:
#load("combinedMeta.Robject")

In [47]:
#make a combined Robject with all datasets in a standardized format for use with splitXAanalysis
combinedCounts <-list()

#'rivron'
combinedCounts$rivron <- list()
combinedCounts$rivron$counts <- rivronLoaded
combinedCounts$rivron$annotation <- combinedMeta[which(combinedMeta$study=="rivron"),]

#'petropolous'
combinedCounts$petropolous <- list()
combinedCounts$petropolous$counts <- petropolous
combinedCounts$petropolous$annotation <- combinedMeta[which(combinedMeta$study=="petropolous"),]

#tyser
combinedCounts$tyser <- list()
combinedCounts$tyser$counts <- tyser
combinedCounts$tyser$annotation <- combinedMeta[which(combinedMeta$study=="tyser"),]

#Pasque
combinedCounts$pasque <- list(counts=pasque)


In [21]:
save(combinedCounts, file="integratedBlastoids.combinedCounts.raw.Robject")

In [5]:
load("integratedBlastoids.combinedCounts.raw.Robject")

In [16]:
loadNormalSeurat <- function(input, projName){
    outSeurat <- CreateSeuratObject(counts=input, project=projName, min.cells=3, min.features=200)#trying min cells 0, was 3 but want to keep more features
    outSeurat <- NormalizeData(outSeurat)
    outSeurat <- FindVariableFeatures(outSeurat, selection.method = "vst", nfeatures = 2000)
    
    outSeurat@meta.data$study <- NA
    outSeurat@meta.data$sampleID <- NA
    outSeurat@meta.data$time <- NA

    for(i in 1:nrow(outSeurat@meta.data)){
        outSeurat@meta.data$study[i] <- combinedMeta$study[which(combinedMeta$cell==rownames(outSeurat@meta.data)[i])]
    outSeurat@meta.data$sampleID[i] <- combinedMeta$sampleID[which(combinedMeta$cell==rownames(outSeurat@meta.data)[i])]
    outSeurat@meta.data$time[i] <- combinedMeta$time[which(combinedMeta$cell==rownames(outSeurat@meta.data)[i])]
    outSeurat@meta.data$Lineage[i] <- combinedMeta$Lineage[which(combinedMeta$cell==rownames(outSeurat@meta.data)[i])]
}
    return(outSeurat)
}

In [50]:
#make seurat objects for everything
indSeurats <- list()

indSeurats[[1]] <- loadNormalSeurat(rivronLoaded, "rivron")
indSeurats[[2]] <- loadNormalSeurat(petropolous, "petropolous")
indSeurats[[3]] <- loadNormalSeurat(tyser, "tyser")
indSeurats[[4]] <- loadNormalSeurat(pasque, "pasque")

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"The following arguments are not used: row.names"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


ERROR: Error in CreateAssayObject(counts = counts, min.cells = min.cells, min.features = min.features, : No cell names (colnames) names present in the input matrix


In [25]:
save(indSeurats, file="allDatasets.indSeurats.Robject")

In [6]:
load("allDatasets.indSeurats.Robject")